# Recording and replaying iMD trajectories with NanoVer

Although the primary benefit of iMD-VR is the ability to simultaneously visualise and interact with molecular systems, it is often useful to save the trajectory produced in order to run analyses after the simulation is complete. This tutorial demonstrates how to use the in-built functionality of NanoVer to record trajectories from iMD simulations, and how to subsequently replay them



## Recording with NanoVer

We will follow a similar setup to our [getting started](getting_started.ipynb) tutorial notebook, using a pre-prepared methane & nanotube input file to run an iMD simulation using OpenMM as the physics engine. Let's start by importing the classes necessary to run the simulation.

In [1]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

Next, let's create an `OpenMMSimulation` object using our pre-prepared input file:

In [2]:
# Create the simulation object
nanotube_simulation = OpenMMSimulation.from_xml_path("openmm_files/nanotube.xml")

As we are recording the data from this simulation, let's set the simulation to deliver the forces and velocities of the system in addition to the default data sent in the frames to the client. This can be achieved by setting the `include_forces` and `include_velocities` flags to `True`, as follows:

In [3]:
# Include forces and velocities
nanotube_simulation.include_forces = True
nanotube_simulation.include_velocities = True

Now we can set everything up to run the simulation, passing our `OpenMMSimulation` to an `OmniRunner` which will serve the simulation for iMD.

In [4]:
# Define the runner
imd_runner = OmniRunner.with_basic_server(nanotube_simulation, name='recording-server')

So far, the setup is no different from a normal iMD simulation. In order to record the trajectory using NanoVer's in-built recording functionality, we need to import the `nanover.omni.record.record_from_server` function:

In [5]:
from nanover.omni.record import record_from_server

When recording the simulation using NanoVer, we create two files: 
* a trajectory file (with the `.traj` extension) that contains the information from the simulation (i.e. the simulation [frames](../fundamentals/frame.ipynb))
* a state file (with the `.state` extension) that contains the updates to the [shared state](../fundamentals/commands_and_state.ipynb)

More information about the contents of these files can be found on the [Recording data](https://irl2.github.io/nanover-docs/concepts/recording.html) page of the NanoVer documentation, but at this stage these details are unimportant. The bottom line is that recording both of these data streams allows NanoVer to replay the recorded trajectory on a server. 


As the recording function generates files containing the recorded data, we need to define the paths of the trajectory and state files that we want to write to.

In [6]:
# Define the .traj and .state file names and paths
traj_path = 'nanotube_recording.traj'
state_path = 'nanotube_recording.state'

We have already set up the server, so all that remains to do before we run the simulation is to pass the port address of the `OmniRunner` that serves the simulation to the `record_from_server` function **before** we start the simulation.

In [7]:
# Pass the runner to the recording function
record_from_server(f"localhost:{imd_runner.app_server.port}", traj_path, state_path)

# Start the simulation
imd_runner.next()

Great! We are now recording our methane & nanotube simulation that's running on a server! If you've [installed NanoverIMD](https://irl2.github.io/nanover-docs/installation.html#installing-the-imd-vr-client), you can connect to the server and interact with the system in VR. Once you've opened the application, click `autoconnect` either from inside VR or at the top left of your on-screen menu, and you'll see something like this:

![nanotube](images/nanover_nanotube.png)

Have a play around and see if you can thread the methane through the nanotube!

Once we're finished with the simulation, we terminate the runner as usual, which also stops the recording.

In [8]:
# Close the runner
imd_runner.close()

Now we have our recorded simulation! 

## Replaying with NanoVer

The benefit of recording a simulation using NanoVer's in-build recording functionality is that it allows us to save the simulation output and load the simulation back into NanoVer directly to visualise the recorded trajectory, while simultaneously playing back the shared state (synchronised with the trajectory), enabling you to e.g. visualise the avatars interacting with the system in VR. If you interacted with the system in VR in the previous section, you'll be able to see yourself interacting with the recorded simulation in this section!


Having recorded a simulation using NanoVer's in-built functionality for recording, the recording can be passed to an `OmniRunner`, which can serve the recorded simulation, enabling multiple clients to connect to the server to visualise it. In this section we'll load our simulation that we recorded above (see previous section!) back into NanoVer. 
 
**NOTE**: This section will not work unless you have _already recorded a simulation_ (see "Recording with NanoVer")

In order to do this, we must first import the `PlaybackSimulation` class:

In [9]:
from nanover.omni.playback import PlaybackSimulation

Now we can create an instance of the `PlaybackSimulation` class from the trajectory and state files created in the previous section for our nanotube simulation, and serve it using an `OmniRunner`. The process is very similar to setting up a regular NanoVer OpenMM simulation from an XML file.

In [10]:
# Create the PlaybackSimulation object
nanotube_recording = PlaybackSimulation(name="nanotube-recording",
                                        traj=traj_path,
                                        state=state_path)

# Pass the PlaybackSimulation to a runner and play the recording
recording_runner = OmniRunner.with_basic_server(nanotube_recording, name="recording-server")
recording_runner.next()

And that's it! Now the recording is up and running on a NanoVer server, and can be visualised by connecting a client to this server. If you've [installed NanoverIMD](https://irl2.github.io/nanover-docs/installation.html#installing-the-imd-vr-client), you can connect to the server and interact with the system in VR. Alternatively, you can connect to the server using the `NGLClient` in our [example notebook](nanover_nglview.ipynb) to view the recorded simulation using and NGLView widget.

As ever, once we are finished with the server playing the recording, we should terminate the runner in the usual fashion.

In [11]:
# Close the runner
recording_runner.close()

That's it! In this notebook we've learned:

* How to record an iMD simulation using NanoVer
* How to replay a recorded iMD simulation using NanoVer

## Next steps

* Check out our [nanotube tutorial](../openmm/openmm_nanotube.ipynb) to learn how to interact with an OpenMM simulation via a python client
* Learn how to change the visual representation of a [protein-ligand system](../openmm/openmm_neuraminidase.ipynb)
* Learn how trajectories recorded directly with NanoVer can be [analysed using MDAnalysis](../mdanalysis/mdanalysis_nanover_recording.ipynb)